<a href="https://colab.research.google.com/github/nabroo101/deep-learning-challenge/blob/main/Charity_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

---

# **Charity Deep Learning Project**

In this project, we'll be exploring deep learning techniques to understand and make predictions related to charity data.

## **Step 1: Importing Dependencies**

Before we dive into the actual data analysis, let's import all the necessary libraries and modules:

```python
# Data manipulation and splitting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Deep learning library
import tensorflow as tf
```

## **Step 2: Loading the Data**

We'll be using a dataset named `charity_data.csv` which contains relevant information about different charities. Let's load this data and take a quick look at the first few rows:

```python
# Import the dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Display the first few rows of the dataset
application_df.head()
```

---




# Alphabet Soup's Dataset Information

From Alphabet Soup’s business team, we have obtained a comprehensive CSV dataset. This dataset encompasses data from over 34,000 organizations that have been beneficiaries of funding from Alphabet Soup throughout the years.

## Columns Description:

The dataset provides a range of columns, capturing metadata about each organization:

- **EIN and NAME**: Identification columns.
  
- **APPLICATION_TYPE**: Describes the Alphabet Soup application type.

- **AFFILIATION**: Represents the affiliated sector of the industry.

- **CLASSIFICATION**: Refers to the government organization classification.

- **USE_CASE**: Specifies the use case for which the funding is granted.

- **ORGANIZATION**: Type of the organization.

- **STATUS**: Indicates whether the organization is currently active or not.

- **INCOME_AMT**: Categorizes organizations based on their income.

- **SPECIAL_CONSIDERATIONS**: Notes if there are any special considerations for the application.

- **ASK_AMT**: The amount of funding the organization requested.

- **IS_SUCCESSFUL**: Determines whether the funding was utilized effectively.




In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import seaborn as sns


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()
print(application_df.info())
application_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB
None


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1




## Data Cleaning

### Dropping Non-Beneficial Columns

For our analysis, certain columns such as 'EIN' and 'NAME' are not beneficial. These columns serve as identification for the organizations but do not offer any substantive information for our machine learning model. Therefore, we'll drop these columns from our dataset:

```python
application_df.drop(columns=["EIN", "NAME"], inplace=True)
```

**Note**: It's essential to include the `inplace=True` parameter if you wish to modify the `application_df` directly. Without it, the changes won't be saved to the `application_df` but will return a new DataFrame with the columns dropped.



In [2]:
dff = application_df["CLASSIFICATION"].value_counts()
dff

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN" , "NAME"] , inplace= True)

#check data
application_df.head()

#check target: to see if there are any bug difrenceses in value count
application_df["IS_SUCCESSFUL"].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64


## Unique Value Analysis

To understand the diversity of our dataset and to ascertain potential candidates for one-hot encoding (among other preprocessing steps), we first determine the number of unique values in each column:

```python
application_df.nunique()
```

By examining the unique count, we can get insights into the categorical nature of our data and decide on the next preprocessing steps.


In [4]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Determine the number of unique values in each column.
application_df_count = application_df.nunique()
application_df_count

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count =  application_df["APPLICATION_TYPE"].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64



1. **Choosing a Cutoff**:
    ```python
    application_types_to_replace = application_type_count[8:].index
    ```
   Here, the code selects a cutoff based on the index. The `[8:]` slice means you're taking all application types from the 8th index onward. This assumes that `application_type_count` is already sorted in descending order by count. The code is effectively saying, "Let's replace all application types from the 8th least frequent type and beyond."

2. **Replacement**:
    ```python
    for app in application_types_to_replace:
        application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    ```
    This loop replaces all the application types in the `application_types_to_replace` list with the label "Other" in the DataFrame `application_df`.

3. **Checking**:
    ```python
    application_df['APPLICATION_TYPE'].value_counts()
    ```
    This line checks and displays the updated counts for each `APPLICATION_TYPE`, verifying that the replacement was successful.



---

### Explanation

---

## **Minimizing Application Types**

In our dataset, there are multiple `APPLICATION_TYPE`s, some of which occur very infrequently. Having many infrequent categories can sometimes introduce noise into our machine learning models, making them less effective. By reducing the number of infrequent categories, we aim to create a more generalizable model.

### **Procedure**:

1. **Determine Cutoff**:
   
   We choose a cutoff to determine which application types are considered "infrequent." For our current approach, we select all application types from the 8th least frequent type onward. These types will be replaced with a more general label: "Other."

    ```python
    application_types_to_replace = application_type_count[8:].index
    ```

2. **Replacement**:
   
   Next, we iterate over the identified application types and replace them in our dataset.

    ```python
    for app in application_types_to_replace:
        application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    ```

3. **Validation**:
   
   Finally, to ensure our changes have taken effect, we check the updated value counts for `APPLICATION_TYPE`.

    ```python
    application_df['APPLICATION_TYPE'].value_counts()
    ```

### **Rationale**:

Minimizing the number of categories in our dataset can:

- **Enhance Model Generalization**: By reducing sparse categories, we reduce the chance of overfitting to specific categories that don't have enough representation.
  
- **Streamline Encoding**: Later, when we encode these categories for modeling, fewer categories can lead to fewer columns, making our dataset more manageable and the model faster.

- **Improve Interpretability**: Models become more interpretable with fewer categories as the focus shifts to more significant, broad categories rather than nuanced, infrequent ones.



In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [application_type_count[application_type_count.values < 500].index]
application_types_to_replace
#review video


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
# application_types_to_replace


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()
classification_count.head(15)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1



## **Minimizing Classifications**

To reduce complexity in the dataset and possibly improve the model's performance, classifications that occur infrequently are grouped together under a single "Other" category.

### **Procedure**:

1. **Determine Classifications to Replace**:
    - Choose classifications that occur less than 100 times.
    ```python
    classification_to_replace = classification_count[classification_count.values < 100].index
    ```

2. **Replace in DataFrame**:
    - Replace the identified classifications in the `CLASSIFICATION` column with "Other."
    ```python
    for cls in classification_to_replace:
        application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    ```

3. **Validate the Replacement**:
    - Verify the changes by checking the updated value counts.
    ```python
    application_df['CLASSIFICATION'].value_counts()
    ```

### **Benefits**:

- **Model Simplicity**: By reducing sparse categories, the model might be less prone to overfitting.
- **Interpretability**: Grouping infrequent classifications into a broader category simplifies interpretation.
- **Computational Efficiency**: Fewer unique values might lead to a more efficient encoding process later in the data preprocessing pipeline.


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_to_replace = classification_count[classification_count.values < 100 ].index
classification_to_replace
# Replace in dataframe
for cls in classification_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns="IS_SUCCESSFUL")
X.SHAPE
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

AttributeError: ignored

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
#  YOUR CODE GOES HERE

In [ ]:
# Train the model
#  YOUR CODE GOES HERE

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE